### [Github Link of repo](https://github.com/akshay-venur/Aidetic-Repo)

### Connecting to the MySQL database using JDBC driver

In [0]:
# JDBC driver class which will be used to connect to the MySQL DB
driver = "org.mariadb.jdbc.Driver"

# Azure MySQL server connection details
database_host = "azure-mysql-server-27-11.mysql.database.azure.com" # servername of the Azure Mysql server
database_port = "3306"
database_name = "pysparkdb" #database name where earthquake table data is present
table = "earthquakedata" # table name where earthquake data is present.
user = "akshayadmin"
password = "mysql@123"

# Constructing the JDBC URL
url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

# Reading data from the specified MySQL table using PySpark and store those data in the pyspark dataframe
df = (spark.read
      .format("jdbc")
      .option("driver", driver)
      .option("url", url)
      .option("dbtable", table)
      .option("user", user)
      .option("password", password)
      .load()
)


### Importing all the required libraries

In [0]:
from pyspark.sql.functions import col, to_date , dayofweek ,date_format  , dayofmonth, month, year, round, avg, stddev, count, corr

###  Q1. How does the Day of a Week affect the number of earthquakes?

In [0]:
#filter only those records whose 'Type' is equal to 'Earthquake'
earthquake_df = df.filter(col("Type") == "Earthquake")

# Extract the day of the week from the "Date" column
earthquake_df = earthquake_df.withColumn("DayOfWeek", dayofweek(col("Date")))


# Group by day of the week and count the occurrences and order by data based on descending order of count
result_df = earthquake_df.groupBy("DayOfWeek").count().orderBy("count", ascending=False)

result_df.show()
# 1-sunday , 2-Monday so on 

+---------+-----+
|DayOfWeek|count|
+---------+-----+
|        7| 3402|
|        4| 3399|
|        6| 3343|
|        2| 3339|
|        5| 3274|
|        1| 3241|
|        3| 3231|
+---------+-----+



In [0]:
# Instaed of week number if we want week name we can use this logic
earthquake_df = df.filter(col("Type") == "Earthquake")

# Extract the name of the week from the "Date" column
earthquake_df = earthquake_df.withColumn("DayOfWeek", date_format(col("Date"), "EEEE"))

# Group by day of the week and count the occurrences and order the data based on descending order of count
result_df = earthquake_df.groupBy("DayOfWeek").count().orderBy("count", ascending=False)

display(result_df)

DayOfWeek,count
Saturday,3402
Wednesday,3399
Friday,3343
Monday,3339
Thursday,3274
Sunday,3241
Tuesday,3231


Databricks visualization. Run in Databricks to view.

### Q2. What is the relation between Day of the month and Number of earthquakes that happened in a year?

In [0]:
# Extract the day of the month from the "Date" column
earthquake_df = earthquake_df.withColumn("DayOfMonth", dayofmonth(col("Date")))

# Group by the day of the month and count the occurrences and  order the data based on descending order of count
result_df = earthquake_df.groupBy("DayOfMonth").count().orderBy("count", ascending=False)

# Display the result DataFrame
display(result_df)

DayOfMonth,count
11,900
17,840
23,830
20,819
16,801
14,796
28,794
26,791
12,785
7,778


Databricks visualization. Run in Databricks to view.

### Q3. What does the average frequency of earthquakes in a month from the year 1965 to 2016 tell us?

In [0]:
# Filter records for earthquakes only and within the specified date range
earthquake_df = df.filter((col("Type") == "Earthquake") & (year("Date").between(1965, 2016)))

# Extract the year and month from the "Date" column
earthquake_df = earthquake_df.withColumn("Year", year(col("Date")))
earthquake_df = earthquake_df.withColumn("MonthName", date_format(col("Date"), "MMMM"))
earthquake_df = earthquake_df.withColumn("Month", month(col("Date")))

# Group by year and month, count the occurrences
result_df = earthquake_df.groupBy("Year", "MonthName").count().orderBy("Year", "MonthName")

# Calculate the average frequency of earthquakes per month
avg_df = result_df.groupBy("MonthName").agg({"count": "avg"})

# Sort the result DataFrame by descending average frequency
sorted_avg_df = avg_df.orderBy(col("avg(count)").desc())

# Display the result DataFrame and average frequency
display(sorted_avg_df)


MonthName,avg(count)
March,40.44230769230769
August,38.46153846153846
December,38.01923076923077
September,37.92307692307692
November,37.88461538461539
April,37.65384615384615
May,37.40384615384615
October,37.0
January,36.30769230769231
July,35.86538461538461


Databricks visualization. Run in Databricks to view.

### Q4. What is the relation between Year and Number of earthquakes that happened in that year?

In [0]:
#filter only those records whose 'Type' is equal to 'Earthquake'
earthquake_df = df.filter((col("Type") == "Earthquake"))

# Extract the year from the "Date" column
earthquake_df = earthquake_df.withColumn("Year", year(col("Date")))

# Group by year, count the occurrences order the data based on descending order of count
result_df = earthquake_df.groupBy("Year").count().orderBy("count", ascending=False)
display(result_df)

Year,count
2011,712
2007,608
1995,589
2004,571
2010,559
2000,553
1996,540
2005,533
1992,532
1990,524


Databricks visualization. Run in Databricks to view.

### Q5. How has the earthquake magnitude on average been varied over the years?

In [0]:

# Filter records for earthquakes only
earthquake_df = df.filter((col("Type") == "Earthquake"))

# Extract the year from the "Date" column
earthquake_df = earthquake_df.withColumn("Year", year(col("Date")))

# Group by year, calculate the average magnitude, and alias the result column
result_df = earthquake_df.groupBy("Year").agg(avg("Magnitude").alias("AverageMagnitude")).orderBy("Year")

# Print a message
print("Average Earthquake Magnitude Over the Years:")

# Display the result DataFrame showing the average earthquake magnitude per year
display(result_df)


Average Earthquake Magnitude Over the Years:


Year,AverageMagnitude
1965,6.0141592920354
1966,6.042274678111588
1967,6.003921568627454
1968,6.081848184818479
1969,6.009937888198749
1970,6.037209302325578
1971,5.972538860103624
1972,5.943814432989688
1973,5.807614213197964
1974,5.828591549295768


Databricks visualization. Run in Databricks to view.

### Q6. How does year impact the standard deviation of the earthquakes?

In [0]:

# Assuming 'df' is your PySpark DataFrame with the specified columns

# Extract the year from the "Date" column
df = df.withColumn("Year", year(col("Date")))

# Group by year and calculate the standard deviation of Magnitude for each year
std_dev_df = df.groupBy("Year").agg(stddev("Magnitude").alias("MagnitudeStdDev"))

# Display the result DataFrame
display(std_dev_df)


Year,MagnitudeStdDev
1990,0.4217373791625804
1975,0.4262235905791955
1977,0.376985237157115
2003,0.4364291266069527
2007,0.4453885403037084
1974,0.39486411203660154
2015,0.4636753312589082
2006,0.3994639806481321
1978,0.40188772760810154
2013,0.45637261052134354


Databricks visualization. Run in Databricks to view.

### Q7. Does geographic location have anything to do with earthquakes?

In [0]:
# Filter records to include only earthquakes
earthquake_df = df.filter((col("Type") == "Earthquake"))

# Group by latitude and longitude, count the occurrences, and order by count in descending order
result_df = earthquake_df.groupBy('Latitude', 'Longitude').count().orderBy("count", ascending=False)

# Display the result DataFrame showing the count of earthquakes at each latitude and longitude
display(result_df)


Latitude,Longitude,count
51.5,-174.8,4
34.416,-118.37,3
38.64,142.75,2
3.781,-32.002,1
27.794,54.879,1
-16.462,-174.742,1
51.3674,-175.2847,1
33.167,137.944,1
-31.193,-13.338,1
-61.6949,154.6743,1


### Q8. Where do earthquakes occur very frequently?

In [0]:
# Filter the DataFrame to include only earthquake records
earthquake_df = df.filter((col("Type") == "Earthquake"))

# Group by the "Location_Source" column, count the occurrences, and order by count in descending order
result_df = earthquake_df.groupBy('Location_Source').count().orderBy("count", ascending=False)

# Display the result DataFrame showing the count of earthquakes for each location source
display(result_df)


Location_Source,count
US,20179
ISCGEM,2581
GCMT,56
NC,54
CI,49
GUC,46
AEIC,40
UNM,21
PGC,19
WEL,18


### Q9. What is the relation between Magnitude, Magnitude Type , Status and Root Mean Square of the earthquakes?

In [0]:
#Relationship between Magnitude and the Magnitude type

# Filter the DataFrame to include only earthquake records
earthquake_df = df.filter((col("Type") == "Earthquake"))

# Group by Magnitude Type, calculate the average Magnitude and count for each type
relationship_df = earthquake_df.groupBy("Magnitude_Type").agg(
    avg("Magnitude").alias("AverageMagnitude"),
    count("*").alias("Count")
)

# Sort the result DataFrame by descending average magnitude
sorted_relationship_df = relationship_df.orderBy(col("AverageMagnitude").desc())

# Display the sorted result DataFrame
display(sorted_relationship_df)


Magnitude_Type,AverageMagnitude,Count
MH,6.540000000000001,5
MWW,6.008673726676766,1983
MS,5.994359576968296,1702
MD,5.966666666666668,6
MW,5.9338375858048105,7721
MWB,5.907282343368652,2458
MWC,5.858125992588608,5667
ML,5.842461538461538,65
null,5.706666666666666,3
MB,5.674561647648214,3593


Databricks visualization. Run in Databricks to view.

In [0]:
#Relationship between Magnitude and the status

# Filter the DataFrame to include only earthquake records
earthquake_df = df.filter((col("Type") == "Earthquake"))

# Group by Magnitude Type, calculate the average Magnitude and count for each type
relationship_df = earthquake_df.groupBy("Status").agg(
    avg("Magnitude").alias("AverageMagnitude"),
    count("*").alias("Count")
)

# Sort the result DataFrame by descending average magnitude
sorted_relationship_df = relationship_df.orderBy(col("AverageMagnitude").desc())

# Display the sorted result DataFrame
display(sorted_relationship_df)

Status,AverageMagnitude,Count
Automatic,6.005615763546828,2639
Reviewed,5.867048081593184,20590


In [0]:
#Relationship between Magnitude and the Root mean sqaure of the earthquake

# Correlation
correlation = df.select(corr("Magnitude", "Root_Mean_Square").alias("Correlation"))


print("Correlation between Magnitude and Root Mean Square:")
display(correlation)

Correlation between Magnitude and Root Mean Square:


Correlation
0.07592836140104058
